# Table Extraction with [PDF Plumber](https://github.com/jsvine/pdfplumber)

[PPAC Fuel prices PDF](https://ppac.gov.in/uploads/page-images/1768544985_PP_9_a_DailyPriceMSHSD_Metro_16.01.2026.pdf)

In [36]:
import pdfplumber
import pandas as pd
import re
from io import StringIO
from tqdm import tqdm

In [3]:
pdf_path = "PPAC_Prices_Fuel.pdf"

In [16]:
pdf = pdfplumber.open(pdf_path)

In [13]:
pdf.metadata

{'Author': 'Anivesh Prasad',
 'CreationDate': "D:20260116095850+05'30'",
 'ModDate': "D:20260116095850+05'30'",
 'Producer': 'Microsoft® Excel® for Microsoft 365',
 'Creator': 'Microsoft® Excel® for Microsoft 365'}

In [21]:
pdf.pages # Returns list of pages

[<Page:1>,
 <Page:2>,
 <Page:3>,
 <Page:4>,
 <Page:5>,
 <Page:6>,
 <Page:7>,
 <Page:8>,
 <Page:9>,
 <Page:10>,
 <Page:11>,
 <Page:12>,
 <Page:13>,
 <Page:14>,
 <Page:15>,
 <Page:16>,
 <Page:17>,
 <Page:18>,
 <Page:19>,
 <Page:20>,
 <Page:21>,
 <Page:22>,
 <Page:23>,
 <Page:24>,
 <Page:25>,
 <Page:26>,
 <Page:27>,
 <Page:28>,
 <Page:29>,
 <Page:30>,
 <Page:31>,
 <Page:32>,
 <Page:33>,
 <Page:34>,
 <Page:35>]

Each of the above `<Page>` element in the list above is the core class of pdfplumber. Most operations we'd do are with this. The main opertaion froo table extraction is `extract_table()`. It'll return the rows in a list.

In [32]:
rows1 = pdf.pages[1].extract_table()
rows1

[['Petroleum Planning & Analysis Cell\nTable Posted: 16-Jan-26',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 ['',
  'Retail Selling Price of Petrol',
  None,
  None,
  None,
  None,
  '',
  'Retail Selling Price of Diesel',
  None,
  None,
  None,
  None,
  ''],
 ['', None, None, None, None, None, None, None, None, None, None, None, None],
 ['',
  'Date of Revision',
  'Delhi',
  'Mumbai',
  'Chennai',
  'Kolkata',
  '',
  'Date of Revision',
  'Delhi',
  'Mumbai',
  'Chennai',
  'Kolkata',
  ''],
 [None,
  None,
  '(Rs./Litre)',
  None,
  None,
  None,
  None,
  None,
  '(Rs./Litre)',
  None,
  None,
  None,
  None],
 ['',
  '9-Oct-25',
  '9 4.77',
  '1 03.50',
  '1 00.80',
  '1 05.41',
  '',
  '9-Oct-25',
  '87.67',
  '9 0.03',
  '92.39',
  '92.02',
  ''],
 [None,
  '8-Oct-25',
  '9 4.77',
  '1 03.50',
  '1 00.80',
  '1 05.41',
  None,
  '8-Oct-25',
  '87.67',
  '9 0.03',
  '92.39',
  '92.02',
  None],
 [None,
  '7-Oct-25',
  '9 4

We can observe that the required data is from 6th row onwards. It'll require some cleaning. 

In [33]:
rows1[5]

['',
 '9-Oct-25',
 '9 4.77',
 '1 03.50',
 '1 00.80',
 '1 05.41',
 '',
 '9-Oct-25',
 '87.67',
 '9 0.03',
 '92.39',
 '92.02',
 '']

Let's ensure that 7th row is the beginning of required data in other pages as well.

In [34]:
rows6 = pdf.pages[6].extract_table()
rows6[5]

['',
 '21-Jun-24',
 '9 4.72',
 '1 04.21',
 '1 00.75',
 '1 03.94',
 '',
 '21-Jun-24',
 '87.62',
 '9 2.15',
 '92.34',
 '90.76',
 '']

We'll have to separate these data points.
1. date -> 1st element of row
2. petrol price in delhi -> 2nd element of row
3. petrol price in mumbai -> 3rd element of row
4. petrol price in chennai -> 4th element of row
5. petrol price in kolkata -> 5th element of row
6. petrol price in delhi -> 8th element of row
7. petrol price in mumbai -> 9th element of row
8. petrol price in chennai -> 10th element of row
9. petrol price in kolkata -> 11th element of row

In [51]:
dates = []
delhi_petrolPrices = []
mumbai_petrolPrices = []
chennai_petrolPrices = []
kolkata_petrolPrices = []

delhi_dieselPrices = []
mumbai_dieselPrices = []
chennai_dieselPrices = []
kolkata_dieselPrices = []

for page in tqdm(pdf.pages):
    rows = page.extract_table()[5:]
    for row in rows:
        date = row[1]
        petrolprice_delhi = row[2].replace(' ', '')
        petrolprice_mumbai = row[3].replace(' ', '')
        petrolprice_chennai = row[4].replace(' ', '')
        petrolprice_kolkata = row[5].replace(' ', '')

        
        dieselprice_delhi = row[8].replace(' ', '')
        dieselprice_mumbai = row[9].replace(' ', '')
        dieselprice_chennai = row[10].replace(' ', '')
        dieselprice_kolkata = row[11].replace(' ', '')

        dates.append(date)
        delhi_petrolPrices.append(petrolprice_delhi)
        mumbai_petrolPrices.append(petrolprice_mumbai)
        chennai_petrolPrices.append(petrolprice_chennai)
        kolkata_petrolPrices.append(petrolprice_kolkata)

        delhi_dieselPrices.append(dieselprice_delhi)
        mumbai_dieselPrices.append(dieselprice_mumbai)
        chennai_dieselPrices.append(dieselprice_chennai)
        kolkata_dieselPrices.append(dieselprice_kolkata)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [01:29<00:00,  2.57s/it]


In [85]:
petrolprices_df = pd.DataFrame([dates, delhi_petrolPrices, mumbai_petrolPrices, chennai_petrolPrices, kolkata_petrolPrices]).T
petrolprices_df = petrolprices_df[1:]
petrolprices_df.columns = ['date','delhi','mumbai','chennai','kolkata']
petrolprices_df['date'] = pd.to_datetime(petrolprices_df['date'], format="%d-%b-%y")
petrolprices_df['year'] = petrolprices_df['date'].dt.year
petrolprices_df['month'] = petrolprices_df['date'].dt.month
petrolprices_df['week_number'] = petrolprices_df['date'].dt.isocalendar().week
petrolprices_df.to_csv('PetrolPricesPPAC_2026Jan16.csv',index=False)

In [86]:
dieselprices_df = pd.DataFrame([dates, delhi_dieselPrices, mumbai_dieselPrices, chennai_dieselPrices, kolkata_dieselPrices]).T
dieselprices_df = dieselprices_df[1:]
dieselprices_df.columns = ['date','delhi','mumbai','chennai','kolkata']
dieselprices_df['date'] = pd.to_datetime(dieselprices_df['date'], format="%d-%b-%y")
dieselprices_df['year'] = dieselprices_df['date'].dt.year
dieselprices_df['month'] = dieselprices_df['date'].dt.month
dieselprices_df['week_number'] = dieselprices_df['date'].dt.isocalendar().week
dieselprices_df.to_csv('DieselPricesPPAC_2026Jan16.csv',index=False)

In [47]:
f

'103.94'